# 病毒傳播模型


## 設置

1. 地圖大小: m * m

2. 人數: N ，defalut = m * m // 2

3. 人物屬性: N * 7(0編號，1狀態，2行動能力，3治癒能力，4免疫，5該回合是否篩檢過，6是否隔離)

4. 狀態: 0健康，D死亡，0~D帶病原，是上帝視角

5. 軌跡: N * track_day shape array

6. 醫院容量 hospital_c

7. 醫療人員治癒能力: cure

8. 快篩檢測能力: P(陽|Yes) = 0.95 = P_py; P(陰|No) = 0.95 = P_nn

9. 能檢測出的threshold，Threshold_detect

10. 有病徵的threshold，Threshold_o

Threshold_o > Threshold_detect

11. 狀態>Th_detect視為Yes

12. 使用常態分布來產生每個人的治癒力，mean = 0.95，std = 0.05

13. 傳染力 = k*狀態 - b 

14. 傳染方法:帶病原者軌跡附近-r ~ +r範圍內，r = 1


## 流程

### initialize and compile():

產生必要固定參數，例如：地圖大小、人口數、醫療容量、死亡標準、檢測標準、檢測準確度、醫療治癒力

隨機初始化人的位置、狀態(0:0.2:0.5 = 9:0.5:0.5)、移動能力(每回合可以朝四周走多遠)、治癒能力(mean=0.95,std=0.05)

先讓每個人按照初始狀態移動track_day天(default=5)

### update_state():更新人物狀態

更新每個人物體內病原體的量 = 前一天的病毒量 * 本身治癒能力(若小於1，會慢慢康復，若大於1會惡化)

若體內病毒量>死亡判定標準，出局

剩下的人根據地圖上的傳染力被感染病毒(若本身是痊癒的人、本身已經帶有病原的人、被隔離不會被重複感染)

若自己康復了，會獲得免疫力，之後也不會在重複感染


### spread_region():帶原者向地圖四周投放感染源

再更新地圖上的傳染力(剛剛是根據上一回合位置感染範圍內的人，感染完後，要重新更新傳染範圍)


### exam():檢測是否帶原

若出現病徵的人(本身病毒含量>設定的閾值,threshold_o)，加入佇列(candicate)

針對這個佇列的人，根據過去track_day天的行蹤，有重疊到任一天的每一個人(沒獲得免疫力的人)都抓去檢驗

若為陽性(有可能是false positve)則再加入佇列，直到盤查到柱列中的最後一位

### move():移動位置

那些在candicate佇列裡的人，如果醫療資源還沒歸零，就丟到醫院，增強他的治癒力；

若醫療資源歸零，只能做隔離，不增加治癒力

那些本身病毒含量歸零且位置在醫院或隔離所的人，重新隨機丟回地圖上，並且要標記獲得免疫力

那些本身病毒含量還沒歸零，但是不在地圖上人，原地待著

那些在地圖上且還活著的人，且沒有被檢測出帶原，根據自身移動能力，朝周圍移動


### 重複update_state ~ move 指定N天